In [32]:
import sqlite3
import pandas as pd
from create_open_library_database import create_connection
import json

In [107]:
conn = create_connection()

def normalize_genre(genre):
    if genre.endswith('.'):
        genre = genre[:-1]
    
    genre = genre.lower()
        
    return genre

def get_genre(title, author):
    cur = conn.cursor()
    cur.execute("select genres from books where title=? collate nocase", [str(title).lower()])
    rows = cur.fetchall()
    if len(rows) != 1:
        return None
    
    genres = json.loads(rows[0][0])
    genres = { normalize_genre(genre) for genre in genres }
    
    if len(genres) != 1:
        return None
    
    genre, = genres
        
    return genre

data = pd.read_csv("gutenberg-with-authors.csv", index_col="gutenberg id")

genres = data[['title', 'authors']].apply(lambda x: get_genre(*x), axis=1)
genres = genres[~genres.isnull()]
conn.close()

2.6.0


In [109]:
genres_frame = genres.to_frame()
genres_frame.columns = ['genres']

data_with_genres = pd.merge(data, genres_frame, how='inner', left_index=True, right_index=True)
data_with_genres.to_csv("gutenberg_with_authors_genres.csv")